In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
import numpy as np
import math
from scipy import stats

In [2]:
%matplotlib inline

In [3]:
warnings.filterwarnings("ignore")

In [4]:
path = './data/'

In [5]:
encodings = ['cp949', 'euc-kr', 'utf-8']

def get_df(name):
    for encoding in encodings:
        try:
            df = pd.read_csv(path + f'{name}', encoding=encoding)
            break
        except UnicodeDecodeError:
            continue
    return df

# 1. 데이터 확인

In [6]:
data = get_df('data.csv')

In [7]:
data.head()

,TAG_MIN,배정번호,건조 1존 OP,건조 2존 OP,건조로 온도 1 Zone,건조로 온도 2 Zone,세정기,소입1존 OP,소입2존 OP,소입3존 OP,...,소입로 CP 값,소입로 CP 모니터 값,소입로 온도 1 Zone,소입로 온도 2 Zone,소입로 온도 3 Zone,소입로 온도 4 Zone,솔트 컨베이어 온도 1 Zone,솔트 컨베이어 온도 2 Zone,솔트조 온도 1 Zone,솔트조 온도 2 Zone
0,2022-01-03 11:22:07,102410,75.6648,30.0155,NaN,NaN,68.4386,72.84030,59.7862,51.71690,...,0.450421,NaN,NaN,859.854,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-03 11:22:08,102410,75.6706,32.2732,NaN,NaN,68.4386,78.44150,61.6286,50.44530,...,0.450356,1.146260e-10,860.338,859.780,860.044,859.786,294.658,272.538,328.734,328.865
2,2022-01-03 11:22:09,102410,75.6776,32.1592,98.8533,99.14600,68.4386,78.10990,61.5414,52.01960,...,0.450341,1.145200e-10,860.338,859.780,859.981,859.724,294.658,272.538,328.734,328.805
3,2022-01-03 11:22:11,102410,75.8656,30.8312,98.7918,99.17675,68.4999,77.50725,60.6663,52.69425,...,0.450201,1.144670e-10,860.338,859.842,859.950,859.599,294.719,272.538,328.674,328.865
4,2022-01-03 11:22:12,102410,73.6468,29.5274,98.7918,99.20750,68.4386,76.02620,61.1634,51.69150,...,0.450235,1.145360e-10,860.351,859.791,859.991,859.731,294.721,272.599,328.740,328.808


In [8]:
column_names_1 = {'배정번호': 'AN', '건조 1존 OP': 'DZ1_OP', '건조 2존 OP': 'DZ2_OP'
                ,'건조로 온도 1 Zone': 'DZ1_TEMP', '건조로 온도 2 Zone': 'DZ2_TEMP', '세정기': 'CLEAN'
                ,'소입1존 OP': 'HDZ1_OP', '소입2존 OP': 'HDZ2_OP', '소입3존 OP': 'HDZ3_OP'
                ,'소입4존 OP': 'HDZ4_OP', '소입로 CP 값': 'HDZ_CP', '소입로 CP 모니터 값': 'HDZ_CPM'
                ,'소입로 온도 1 Zone': 'HDZ1_TEMP', '소입로 온도 2 Zone': 'HDZ2_TEMP', '소입로 온도 3 Zone': 'HDZ3_TEMP'
                ,'소입로 온도 4 Zone': 'HDZ4_TEMP', '솔트 컨베이어 온도 1 Zone': 'SCZ1_TEMP', '솔트 컨베이어 온도 2 Zone': 'SCZ2_TEMP'
                ,'솔트조 온도 1 Zone': 'STZ1_TEMP', '솔트조 온도 2 Zone': 'STZ2_TEMP'}

data.rename(columns=column_names_1, inplace=True)

In [9]:
data.describe()

,AN,DZ1_OP,DZ2_OP,DZ1_TEMP,DZ2_TEMP,CLEAN,HDZ1_OP,HDZ2_OP,HDZ3_OP,HDZ4_OP,HDZ_CP,HDZ_CPM,HDZ1_TEMP,HDZ2_TEMP,HDZ3_TEMP,HDZ4_TEMP,SCZ1_TEMP,SCZ2_TEMP,STZ1_TEMP,STZ2_TEMP
count,2.939722e+06,2.939721e+06,2.939721e+06,2.939606e+06,2.939574e+06,2.939631e+06,2.935434e+06,2.939722e+06,2.939720e+06,2.939719e+06,2.939721e+06,2.939575e+06,2.939592e+06,2.939594e+06,2.939565e+06,2.939552e+06,2.939616e+06,2.939580e+06,2.939513e+06,2.939519e+06
mean,1.284422e+05,6.989404e+01,2.044708e+01,1.000061e+02,1.000198e+02,6.771864e+01,7.564373e+01,5.486239e+01,5.386029e+01,7.108925e+01,4.488618e-01,1.145580e-10,8.592077e+02,8.600021e+02,8.600029e+02,8.600062e+02,2.839963e+02,2.799293e+02,3.318062e+02,3.321773e+02
std,1.263704e+04,4.014802e+00,5.217123e+00,4.360371e-01,3.623526e-01,1.630768e+00,2.516083e+01,4.429079e+00,2.664304e+00,2.556959e+00,1.886477e-02,6.012797e-13,3.647667e+00,5.578480e-01,3.518205e-01,4.551778e-01,9.512770e+00,6.611579e+00,7.827379e-01,8.732977e-01
min,1.024100e+05,4.725320e+01,1.188830e-04,9.734210e+01,9.787060e+01,6.062440e+01,8.500550e-04,8.620010e+00,4.370450e-02,6.244200e-03,5.096370e-03,1.116620e-10,8.402980e+02,8.559290e+02,8.582800e+02,8.579920e+02,2.662300e+02,2.664260e+02,3.281610e+02,3.280730e+02
25%,1.194480e+05,6.842880e+01,1.891760e+01,9.981440e+01,9.989010e+01,6.656940e+01,6.496270e+01,5.332590e+01,5.238910e+01,6.967810e+01,4.484415e-01,1.142615e-10,8.579490e+02,8.597760e+02,8.598290e+02,8.598430e+02,2.747540e+02,2.735020e+02,3.318670e+02,3.321780e+02
50%,1.298890e+05,7.051660e+01,2.129310e+01,1.000020e+02,1.000190e+02,6.769720e+01,8.221040e+01,5.566540e+01,5.388620e+01,7.104540e+01,4.500620e-01,1.145550e-10,8.595750e+02,8.600220e+02,8.600020e+02,8.600000e+02,2.845860e+02,2.800200e+02,3.320170e+02,3.324230e+02
75%,1.391160e+05,7.237810e+01,2.338840e+01,1.001910e+02,1.001610e+02,6.897990e+01,9.536660e+01,5.757330e+01,5.541450e+01,7.247710e+01,4.517070e-01,1.148440e-10,8.602580e+02,8.602490e+02,8.601720e+02,8.601580e+02,2.933430e+02,2.863340e+02,3.321410e+02,3.326260e+02
max,1.480690e+05,8.729950e+01,4.753950e+01,1.024690e+02,1.018430e+02,7.149010e+01,1.000000e+02,7.727090e+01,6.601500e+01,8.739070e+01,9.091110e-01,1.329290e-10,8.772280e+02,8.660340e+02,8.701190e+02,8.821480e+02,2.985300e+02,2.916960e+02,3.327170e+02,3.331790e+02


In [10]:
quality = pd.read_excel(path+'quality.xlsx')

In [11]:
quality.head()

,배정번호,작업일,공정명,설비명,양품수량,불량수량,총수량
0,102410,2022-01-03,열처리,열처리 염욕_1,15160,3,15163
1,102585,2022-01-03,열처리,열처리 염욕_1,29892,10,29902
2,102930,2022-01-04,열처리,열처리 염욕_1,59616,30,59646
3,103142,2022-01-05,열처리,열처리 염욕_1,74730,13,74743
4,103675,2022-01-06,열처리,열처리 염욕_1,14979,2,14981


In [12]:
quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   배정번호    136 non-null    int64         
 1   작업일     136 non-null    datetime64[ns]
 2   공정명     136 non-null    object        
 3   설비명     136 non-null    object        
 4   양품수량    136 non-null    int64         
 5   불량수량    136 non-null    int64         
 6   총수량     136 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 7.6+ KB


In [13]:
column_names_2 = {'배정번호': 'AN', '작업일': 'WD', '공정명': 'PN'
                ,'설비명': 'EN', '양품수량': 'GQ', '불량수량': 'BQ'
                ,'총수량': 'TQ'}

quality.rename(columns=column_names_2, inplace=True)

In [14]:
quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   AN      136 non-null    int64         
 1   WD      136 non-null    datetime64[ns]
 2   PN      136 non-null    object        
 3   EN      136 non-null    object        
 4   GQ      136 non-null    int64         
 5   BQ      136 non-null    int64         
 6   TQ      136 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 7.6+ KB


In [15]:
quality.describe()

,AN,WD,GQ,BQ,TQ
count,136.000000,136,136.000000,136.000000,136.000000
mean,128897.191176,2022-04-21 10:35:17.647058944,45012.301471,15.117647,45027.419118
min,102410.000000,2022-01-03 00:00:00,8412.000000,0.000000,8414.000000
25%,120467.750000,2022-03-15 18:00:00,29736.000000,4.000000,29755.000000
50%,130199.000000,2022-04-23 00:00:00,44003.000000,8.500000,44020.500000
75%,138982.500000,2022-06-03 12:00:00,60120.750000,17.000000,60135.500000
max,148069.000000,2022-07-19 00:00:00,104740.000000,120.000000,104761.000000
std,12403.393924,NaN,25548.197992,18.549657,25554.548710


In [16]:
quality.isnull().sum()

AN    0
WD    0
PN    0
EN    0
GQ    0
BQ    0
TQ    0
dtype: int64

In [17]:
train = get_df('train.csv')

In [18]:
train.head()

,Unnamed: 0,건조 1존 OP_Avg,건조 1존 OP_Std,건조 2존 OP_Avg,건조 2존 OP_Std,건조로 온도 1 Zone_Avg,건조로 온도 1 Zone_Std,건조로 온도 2 Zone_Avg,건조로 온도 2 Zone_Std,세정기_Avg,...,소입로 온도 4 Zone_Std,솔트 컨베이어 온도 1 Zone_Avg,솔트 컨베이어 온도 1 Zone_Std,솔트 컨베이어 온도 2 Zone_Avg,솔트 컨베이어 온도 2 Zone_Std,솔트조 온도 1 Zone_Avg,솔트조 온도 1 Zone_Std,솔트조 온도 2 Zone_Avg,솔트조 온도 2 Zone_Std,불량단계
0,97,69.497726,3.274577,20.310463,3.490991,99.999143,0.435237,100.001123,0.401438,67.864965,...,0.304168,284.699659,9.601850,280.411936,6.940009,332.111266,0.152253,332.712474,0.153026,안정
1,125,68.776700,3.548587,16.547672,4.161717,100.077760,0.394062,100.107134,0.291589,69.614220,...,0.303700,285.007150,9.239152,280.646734,6.650701,332.123215,0.225985,332.093658,0.329912,위험
2,11,73.502913,2.645737,21.218347,2.218216,100.006615,0.387797,99.992686,0.281373,66.220995,...,0.402050,283.120448,9.426413,279.110908,6.064772,332.182887,0.099322,332.407261,0.107440,안정
3,129,68.062513,3.439085,4.366498,3.727635,100.040387,0.418439,100.139576,0.334492,69.242707,...,0.360325,285.074759,9.475964,280.790056,6.699134,332.277923,0.121808,332.261568,0.184196,안정
4,48,68.820299,3.946638,19.902113,3.765778,100.071930,0.367971,100.097453,0.291788,65.512487,...,0.211795,284.166005,9.847216,279.587268,6.606966,331.943223,0.189118,332.503069,0.359428,안정


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             108 non-null    int64  
 1   건조 1존 OP_Avg           108 non-null    float64
 2   건조 1존 OP_Std           108 non-null    float64
 3   건조 2존 OP_Avg           108 non-null    float64
 4   건조 2존 OP_Std           108 non-null    float64
 5   건조로 온도 1 Zone_Avg      108 non-null    float64
 6   건조로 온도 1 Zone_Std      108 non-null    float64
 7   건조로 온도 2 Zone_Avg      108 non-null    float64
 8   건조로 온도 2 Zone_Std      108 non-null    float64
 9   세정기_Avg                108 non-null    float64
 10  세정기_Std                108 non-null    float64
 11  소입1존 OP_Avg            108 non-null    float64
 12  소입1존 OP_Std            108 non-null    float64
 13  소입2존 OP_Avg            108 non-null    float64
 14  소입2존 OP_Std            108 non-null    float64
 15  소입3존 O

In [20]:
column_names_3 = {'Unnamed: 0' : 'UNNAMED', '건조 1존 OP': 'DZ1_OP', '건조 2존 OP': 'DZ2_OP'
                ,'건조로 온도 1 Zone': 'DZ1_TEMP', '건조로 온도 2 Zone': 'DZ2_TEMP', '세정기': 'CLEAN'
                ,'소입1존 OP': 'HDZ1_OP', '소입2존 OP': 'HDZ2_OP', '소입3존 OP': 'HDZ3_OP'
                ,'소입4존 OP': 'HDZ4_OP', '소입로 CP 값': 'HDZ_CP', '소입로 CP 모니터 값 ': 'HDZ_CPM'
                ,'소입로 온도 1 Zone': 'HDZ1_TEMP', '소입로 온도 2 Zone': 'HDZ2_TEMP', '소입로 온도 3 Zone': 'HDZ3_TEMP'
                ,'소입로 온도 4 Zone': 'HDZ4_TEMP', '솔트 컨베이어 온도 1 Zone': 'SCZ1_TEMP', '솔트 컨베이어 온도 2 Zone': 'SCZ2_TEMP'
                ,'솔트조 온도 1 Zone': 'STZ1_TEMP', '솔트조 온도 2 Zone': 'STZ2_TEMP', '불량단계' : 'FS'}

column_names_combined = {}
for key, value in column_names_3.items():
    column_names_combined[key] = value
    column_names_combined[f'{key}_Avg'] = f'{value}_Avg'
    column_names_combined[f'{key}_Std'] = f'{value}_Std'
    
train.rename(columns=column_names_combined, inplace=True)

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   UNNAMED        108 non-null    int64  
 1   DZ1_OP_Avg     108 non-null    float64
 2   DZ1_OP_Std     108 non-null    float64
 3   DZ2_OP_Avg     108 non-null    float64
 4   DZ2_OP_Std     108 non-null    float64
 5   DZ1_TEMP_Avg   108 non-null    float64
 6   DZ1_TEMP_Std   108 non-null    float64
 7   DZ2_TEMP_Avg   108 non-null    float64
 8   DZ2_TEMP_Std   108 non-null    float64
 9   CLEAN_Avg      108 non-null    float64
 10  CLEAN_Std      108 non-null    float64
 11  HDZ1_OP_Avg    108 non-null    float64
 12  HDZ1_OP_Std    108 non-null    float64
 13  HDZ2_OP_Avg    108 non-null    float64
 14  HDZ2_OP_Std    108 non-null    float64
 15  HDZ3_OP_Avg    108 non-null    float64
 16  HDZ3_OP_Std    108 non-null    float64
 17  HDZ4_OP_Avg    108 non-null    float64
 18  HDZ4_OP_St

# 2. 데이터 전처리

## 날짜 변환

In [22]:
data['TAG_MIN'] = pd.to_datetime(data['TAG_MIN'])

In [23]:
data['Year'] = data['TAG_MIN'].dt.year
data['Month'] = data['TAG_MIN'].dt.month
data['Day'] = data['TAG_MIN'].dt.day
data['Hour'] = data['TAG_MIN'].dt.hour
data['Minute'] = data['TAG_MIN'].dt.minute
data['Second'] = data['TAG_MIN'].dt.second

In [24]:
data['Day']

0           3
1           3
2           3
3           3
4           3
           ..
2939717    19
2939718    19
2939719    19
2939720    19
2939721    19
Name: Day, Length: 2939722, dtype: int32

## 결측치 처리

In [25]:
data.isnull().sum()

TAG_MIN         0
AN              0
DZ1_OP          1
DZ2_OP          1
DZ1_TEMP      116
DZ2_TEMP      148
CLEAN          91
HDZ1_OP      4288
HDZ2_OP         0
HDZ3_OP         2
HDZ4_OP         3
HDZ_CP          1
HDZ_CPM       147
HDZ1_TEMP     130
HDZ2_TEMP     128
HDZ3_TEMP     157
HDZ4_TEMP     170
SCZ1_TEMP     106
SCZ2_TEMP     142
STZ1_TEMP     209
STZ2_TEMP     203
Year            0
Month           0
Day             0
Hour            0
Minute          0
Second          0
dtype: int64

In [26]:
data = data.fillna(data.median())

# 3. 데이터 시각화

# 4. 데이터 병합, 종속변수 생성, 차원 축소

## 데이터 병합

In [27]:
columns_to_exclude = ['AN', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']  
included_columns = [col for col in data.columns if col not in columns_to_exclude]

data_stat = data.groupby('AN')[included_columns].agg(['mean', 'std'])
data_stat

TAG_MIN                               DZ1_OP  \
                                mean                       std       mean   
AN                                                                          
102410 2022-01-03 12:24:28.838327296 0 days 00:36:10.264292473  72.252727   
102585 2022-01-04 00:28:00.827881728 0 days 01:05:03.107195301  72.235643   
102930 2022-01-05 02:27:15.832555264 0 days 02:14:36.326628800  70.720207   
103142 2022-01-05 19:06:21.766428416 0 days 02:38:06.365402057  72.424229   
103675 2022-01-06 23:23:03.293663488 0 days 00:31:03.135587680  72.774648   
...                              ...                       ...        ...   
147292 2022-07-15 03:07:57.495409664 0 days 01:38:48.197521852  69.486127   
147546 2022-07-16 04:01:49.968608512 0 days 02:15:53.076850598  69.718808   
147982 2022-07-19 04:21:14.983996160 0 days 01:31:15.405727301  69.799029   
147996 2022-07-19 09:32:31.860800256 0 days 01:19:03.398733136  69.991809   
148069 2022-07-19 15:29:18.065226752 0 days 02:06:54.169444262  69.032823   

                     DZ2_OP              DZ1_TEMP              DZ2_TEMP  \
             std       mean       std        mean       std        mean   
AN                                                                        
102410  3.696537  21.354464  4.348928   99.943476  0.593891  100.061884   
102585  3.365000  18.602563  2.859741   99.987431  0.515429  100.065032   
102930  3.231776  20.911928  2.582097   99.995592  0.472719  100.021641   
103142  2.635245  22.250186  2.402781  100.005055  0.331393  100.009675   
103675  4.159221  21.865151  3.622806   99.983502  0.655347  100.043710   
...          ...        ...       ...         ...       ...         ...   
147292  3.123360   4.376536  2.514362  100.019260  0.381383  100.079487   
147546  2.662344  17.836625  3.098225  100.024207  0.314547  100.005331   
147982  3.164459  17.913929  3.446134  100.028487  0.347958  100.026178   
147996  3.564122  16.868628  5.317683   99.990732  0.381893  100.002368   
148069  3.781360   5.232418  5.488640  100.033525  0.418362  100.249066   

                  ...   HDZ4_TEMP             SCZ1_TEMP             SCZ2_TEMP  \
             std  ...        mean       std        mean       std        mean   
AN                ...                                                           
102410  0.483452  ...  860.010591  0.553425  282.581576  9.371136  280.148995   
102585  0.356142  ...  859.991765  0.480499  282.788156  9.499565  279.772316   
102930  0.343024  ...  860.007487  0.418431  283.330874  9.680441  279.308958   
103142  0.251783  ...  860.003481  0.296744  282.882341  9.495495  279.241106   
103675  0.470749  ...  860.007199  0.571169  283.581648  9.705562  277.544769   
...          ...  ...         ...       ...         ...       ...         ...   
147292  0.273451  ...  859.994794  0.328179  284.629721  9.221702  280.419455   
147546  0.259923  ...  859.998661  0.206355  284.236555  9.317357  279.987627   
147982  0.323286  ...  860.002395  0.313149  284.190848  9.223516  279.977804   
147996  0.378060  ...  859.991821  0.343028  284.781531  9.270745  280.511153   
148069  0.383402  ...  860.004422  0.339789  284.423098  9.276226  280.554417   

                   STZ1_TEMP             STZ2_TEMP            
             std        mean       std        mean       std  
AN                                                            
102410  6.033861  329.016349  0.127638  329.070923  0.122912  
102585  7.161542  328.998615  0.101219  328.924151  0.089118  
102930  6.665152  329.133618  0.121386  329.148777  0.117273  
103142  6.537439  329.082068  0.100404  329.073209  0.102232  
103675  5.365901  329.010901  0.097701  329.114583  0.089317  
...          ...         ...       ...         ...       ...  
147292  6.696057  332.184151  0.110639  332.210159  0.129219  
147546  6.612269  332.155757  0.173016  332.162484  0.260722  
147982  6.537935  332.209271  0.096239  332.125721  0.118037  
147996  

In [28]:
chg_name = {'mean': '_Avg', 'std': '_Std'}
data_stat.columns = list(map(lambda x: x[0] + chg_name[x[1]], data_stat.columns))
data_stat.reset_index(drop=False, inplace=True)

In [29]:
data_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 41 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   AN             136 non-null    int64          
 1   TAG_MIN_Avg    136 non-null    datetime64[ns] 
 2   TAG_MIN_Std    136 non-null    timedelta64[ns]
 3   DZ1_OP_Avg     136 non-null    float64        
 4   DZ1_OP_Std     136 non-null    float64        
 5   DZ2_OP_Avg     136 non-null    float64        
 6   DZ2_OP_Std     136 non-null    float64        
 7   DZ1_TEMP_Avg   136 non-null    float64        
 8   DZ1_TEMP_Std   136 non-null    float64        
 9   DZ2_TEMP_Avg   136 non-null    float64        
 10  DZ2_TEMP_Std   136 non-null    float64        
 11  CLEAN_Avg      136 non-null    float64        
 12  CLEAN_Std      136 non-null    float64        
 13  HDZ1_OP_Avg    136 non-null    float64        
 14  HDZ1_OP_Std    136 non-null    float64        
 15  HDZ2_O

In [30]:
df_total = pd.merge(quality, data_stat, on='AN', how='left')
df_total.head()

,AN,WD,PN,EN,GQ,BQ,TQ,TAG_MIN_Avg,TAG_MIN_Std,DZ1_OP_Avg,...,HDZ4_TEMP_Avg,HDZ4_TEMP_Std,SCZ1_TEMP_Avg,SCZ1_TEMP_Std,SCZ2_TEMP_Avg,SCZ2_TEMP_Std,STZ1_TEMP_Avg,STZ1_TEMP_Std,STZ2_TEMP_Avg,STZ2_TEMP_Std
0,102410,2022-01-03,열처리,열처리 염욕_1,15160,3,15163,2022-01-03 12:24:28.838327296,0 days 00:36:10.264292473,72.252727,...,860.010591,0.553425,282.581576,9.371136,280.148995,6.033861,329.016349,0.127638,329.070923,0.122912
1,102585,2022-01-03,열처리,열처리 염욕_1,29892,10,29902,2022-01-04 00:28:00.827881728,0 days 01:05:03.107195301,72.235643,...,859.991765,0.480499,282.788156,9.499565,279.772316,7.161542,328.998615,0.101219,328.924151,0.089118
2,102930,2022-01-04,열처리,열처리 염욕_1,59616,30,59646,2022-01-05 02:27:15.832555264,0 days 02:14:36.326628800,70.720207,...,860.007487,0.418431,283.330874,9.680441,279.308958,6.665152,329.133618,0.121386,329.148777,0.117273
3,103142,2022-01-05,열처리,열처리 염욕_1,74730,13,74743,2022-01-05 19:06:21.766428416,0 days 02:38:06.365402057,72.424229,...,860.003481,0.296744,282.882341,9.495495,279.241106,6.537439,329.082068,0.100404,329.073209,0.102232
4,103675,2022-01-06,열처리,열처리 염욕_1,14979,2,14981,2022-01-06 23:23:03.293663488,0 days 00:31:03.135587680,72.774648,...,860.007199,0.571169,283.581648,9.705562,277.544769,5.365901,329.010901,0.097701,329.114583,0.089317


In [31]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 47 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   AN             136 non-null    int64          
 1   WD             136 non-null    datetime64[ns] 
 2   PN             136 non-null    object         
 3   EN             136 non-null    object         
 4   GQ             136 non-null    int64          
 5   BQ             136 non-null    int64          
 6   TQ             136 non-null    int64          
 7   TAG_MIN_Avg    136 non-null    datetime64[ns] 
 8   TAG_MIN_Std    136 non-null    timedelta64[ns]
 9   DZ1_OP_Avg     136 non-null    float64        
 10  DZ1_OP_Std     136 non-null    float64        
 11  DZ2_OP_Avg     136 non-null    float64        
 12  DZ2_OP_Std     136 non-null    float64        
 13  DZ1_TEMP_Avg   136 non-null    float64        
 14  DZ1_TEMP_Std   136 non-null    float64        
 15  DZ2_TE

## 종속변수 생성

In [32]:
# 종속변수 생성을 위한 파생변수 생성
df_total["FR"] = round(df_total["BQ"] / df_total["TQ"] *100, 3)

In [33]:
df_total.groupby('AN')['FR'].mean()

AN
102410    0.020
102585    0.033
102930    0.050
103142    0.017
103675    0.013
          ...  
147292    0.027
147546    0.027
147982    0.029
147996    0.030
148069    0.000
Name: FR, Length: 136, dtype: float64

In [34]:
# 6시그마 기준으로 하여 불량여부 판단

def calculate_upper_bound(row):
    return row['FR'] + 3 * math.sqrt((row['FR'] * (1 - row['FR'])) / row['TQ'])

df_total['upper_bound'] = df_total.apply(calculate_upper_bound, axis=1)

def set_fr_level(row):
    if row['FR'] > row['upper_bound']:
        return 'dangerous'
    else:
        return 'stable'

df_total['FR_Level'] = df_total.apply(set_fr_level, axis=1)
df_total = df_total.drop('upper_bound', axis=1)                      

In [35]:
df_total['FR_Level'].value_counts()

FR_Level
stable    136
Name: count, dtype: int64

In [36]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   AN             136 non-null    int64          
 1   WD             136 non-null    datetime64[ns] 
 2   PN             136 non-null    object         
 3   EN             136 non-null    object         
 4   GQ             136 non-null    int64          
 5   BQ             136 non-null    int64          
 6   TQ             136 non-null    int64          
 7   TAG_MIN_Avg    136 non-null    datetime64[ns] 
 8   TAG_MIN_Std    136 non-null    timedelta64[ns]
 9   DZ1_OP_Avg     136 non-null    float64        
 10  DZ1_OP_Std     136 non-null    float64        
 11  DZ2_OP_Avg     136 non-null    float64        
 12  DZ2_OP_Std     136 non-null    float64        
 13  DZ1_TEMP_Avg   136 non-null    float64        
 14  DZ1_TEMP_Std   136 non-null    float64        
 15  DZ2_TE

# 5. 통계분석

In [37]:
df = data.drop(['TAG_MIN', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second'] , axis=1)

In [38]:
quality['BQ_Rate'] = (quality['BQ'] / quality['TQ']) * 100

In [39]:
quality_pcc = quality.drop(['WD', 'PN', 'EN', 'GQ', 'BQ', 'TQ'], axis=1)
quality_pcc

,AN,BQ_Rate
0,102410,0.019785
1,102585,0.033443
2,102930,0.050297
3,103142,0.017393
4,103675,0.013350
...,...,...
131,147292,0.027412
132,147546,0.026679
133,147982,0.029273
134,147996,0.029754


## 설비별 이상치와 불량률과의 PCC

In [40]:
df = data.drop(['TAG_MIN', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second'], axis=1)

In [41]:
for col in df.columns.drop('AN'):
    # Calculate the first and third quartiles for the 'Dry 1-zone OP' column
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)

    # Calculate the IQR for the 'Dry 1-zone OP' column
    iqr = 1.5 * (q3 - q1)

    # Create a boolean mask for outliers in the 'Dry 1-zone OP' column
    outlier_mask = (df[col] < (q1 - iqr)) | (df[col] > (q3 + iqr))

    # Create a new column 'Outlier' to label rows with 1 for outliers and 0 for non-outliers
    df[col] = np.where(outlier_mask, 1, 0)

In [42]:
merged_df1 = quality_pcc.merge(df.groupby('AN').sum(), how='inner', on='AN')

In [43]:
from scipy.stats import pearsonr

correlation_results = {}
for column in merged_df1.columns:
    if column != 'BQ_Rate':
        correlation_coefficient, p_value = pearsonr(merged_df1['BQ_Rate'], merged_df1[column])
        correlation_results[column] = {'correlation_coefficient': correlation_coefficient, 'p_value': p_value}

# 결과 출력
for column, result in sorted(correlation_results.items(), key=lambda x: x[1]['correlation_coefficient'], reverse=True):
    print(f"{column}: correlation_coefficient = {result['correlation_coefficient']:.4f}, p-value = {result['p_value']:.4f}")

HDZ2_TEMP: correlation_coefficient = 0.0673, p-value = 0.4362
HDZ1_TEMP: correlation_coefficient = 0.0536, p-value = 0.5357
HDZ_CPM: correlation_coefficient = 0.0474, p-value = 0.5835
HDZ3_TEMP: correlation_coefficient = 0.0460, p-value = 0.5946
DZ1_TEMP: correlation_coefficient = 0.0315, p-value = 0.7158
AN: correlation_coefficient = 0.0314, p-value = 0.7167
DZ2_TEMP: correlation_coefficient = 0.0289, p-value = 0.7388
HDZ1_OP: correlation_coefficient = 0.0217, p-value = 0.8022
HDZ4_TEMP: correlation_coefficient = 0.0133, p-value = 0.8775
HDZ_CP: correlation_coefficient = -0.0353, p-value = 0.6829
HDZ4_OP: correlation_coefficient = -0.0381, p-value = 0.6601
CLEAN: correlation_coefficient = -0.0563, p-value = 0.5152
DZ2_OP: correlation_coefficient = -0.0617, p-value = 0.4754
DZ1_OP: correlation_coefficient = -0.0744, p-value = 0.3894
HDZ3_OP: correlation_coefficient = -0.0869, p-value = 0.3143
HDZ2_OP: correlation_coefficient = -0.1367, p-value = 0.1126
SCZ1_TEMP: correlation_coefficien

## 설비 총 이상치와 불량률과의 PCC

In [44]:
df = data.drop(['TAG_MIN', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second'], axis=1)

In [45]:
df['Outlier'] = 0

for col in df.columns:
    if col != 'AN':
        Q1 = np.quantile(df[col], 0.25)
        Q3 = np.quantile(df[col], 0.75)
        IQR = Q3 - Q1
        lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

        outliers = (df[col] > upper_bound) | (df[col] < lower_bound)
        df['Outlier'] += outliers.astype(int)

In [46]:
merged_df2 = quality_pcc.merge(df.groupby('AN').sum(), how='inner', on='AN')

In [47]:
for column in merged_df2.columns:
    if column not in ['BQ_Rate', 'Outlier']:
        merged_df2.drop(column, axis=1, inplace=True)

In [48]:
from scipy.stats import pearsonr

correlation_results = {}
for column in merged_df2.columns:
    if column != 'BQ_Rate':
        correlation_coefficient, p_value = pearsonr(merged_df2['BQ_Rate'], merged_df2[column])
        correlation_results[column] = {'correlation_coefficient': correlation_coefficient, 'p_value': p_value}

# 결과 출력
for column, result in sorted(correlation_results.items(), key=lambda x: x[1]['correlation_coefficient'], reverse=True):
    print(f"{column}: correlation_coefficient = {result['correlation_coefficient']:.4f}, p-value = {result['p_value']:.4f}")

Outlier: correlation_coefficient = -0.1140, p-value = 0.1864


## (설비데이터&이상치) BQ와의 PCC

In [49]:
df = data.drop(['TAG_MIN', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second'], axis=1)

In [50]:
df['Outlier'] = 0

for col in df.columns:
    if col != 'AN':
        Q1 = np.quantile(df[col], 0.25)
        Q3 = np.quantile(df[col], 0.75)
        IQR = Q3 - Q1
        lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

        outliers = (df[col] > upper_bound) | (df[col] < lower_bound)
        df['Outlier'] += outliers.astype(int)

In [51]:
merged_df3 = quality.drop(['WD', 'PN', 'EN', 'GQ', 'TQ', 'BQ_Rate'], axis=1).merge(df.groupby('AN').mean(), how='inner', on='AN')

In [52]:
from scipy.stats import pearsonr

correlation_results = {}
for column in merged_df3.columns:
    if column != 'BQ':
        correlation_coefficient, p_value = pearsonr(merged_df3['BQ'], merged_df3[column])
        correlation_results[column] = {'correlation_coefficient': correlation_coefficient, 'p_value': p_value}

# 결과 출력
for column, result in sorted(correlation_results.items(), key=lambda x: x[1]['correlation_coefficient'], reverse=True):
    print(f"{column}: correlation_coefficient = {result['correlation_coefficient']:.4f}, p-value = {result['p_value']:.4f}")

HDZ1_OP: correlation_coefficient = 0.1838, p-value = 0.0322
STZ2_TEMP: correlation_coefficient = 0.1207, p-value = 0.1615
SCZ1_TEMP: correlation_coefficient = 0.1094, p-value = 0.2050
HDZ2_OP: correlation_coefficient = 0.1065, p-value = 0.2171
STZ1_TEMP: correlation_coefficient = 0.1002, p-value = 0.2457
HDZ_CP: correlation_coefficient = 0.0977, p-value = 0.2579
HDZ4_OP: correlation_coefficient = 0.0917, p-value = 0.2885
AN: correlation_coefficient = 0.0545, p-value = 0.5284
DZ2_OP: correlation_coefficient = 0.0421, p-value = 0.6267
SCZ2_TEMP: correlation_coefficient = 0.0255, p-value = 0.7684
HDZ3_OP: correlation_coefficient = 0.0178, p-value = 0.8372
DZ1_OP: correlation_coefficient = 0.0118, p-value = 0.8916
DZ1_TEMP: correlation_coefficient = 0.0083, p-value = 0.9237
CLEAN: correlation_coefficient = 0.0028, p-value = 0.9746
DZ2_TEMP: correlation_coefficient = -0.0034, p-value = 0.9683
HDZ2_TEMP: correlation_coefficient = -0.0593, p-value = 0.4927
HDZ4_TEMP: correlation_coefficient =

## 불량률 차이가 많이나는 두 그룹간의 T-test

In [53]:
df = data.drop(['TAG_MIN', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second'], axis=1)

In [54]:
merged_df4 = quality_pcc.merge(df.groupby('AN').mean(), how='inner', on='AN')

In [55]:
merged_df4['BQ_Rate'].idxmax()

63

In [56]:
merged_df4['BQ_Rate'].idxmin()

6

In [57]:
result = merged_df4.iloc[[6, 63]]

In [58]:
result

,AN,BQ_Rate,DZ1_OP,DZ2_OP,DZ1_TEMP,DZ2_TEMP,CLEAN,HDZ1_OP,HDZ2_OP,HDZ3_OP,...,HDZ_CP,HDZ_CPM,HDZ1_TEMP,HDZ2_TEMP,HDZ3_TEMP,HDZ4_TEMP,SCZ1_TEMP,SCZ2_TEMP,STZ1_TEMP,STZ2_TEMP
6,104126,0.000000,73.328862,22.591392,100.047576,100.060726,69.898156,63.774512,44.791223,47.318476,...,0.341091,1.146382e-10,860.259000,860.055945,860.050157,860.003010,282.642287,278.764513,328.831918,328.945645
63,128795,0.368509,68.056119,20.104963,99.973817,100.007704,66.168607,77.392989,54.090259,52.985615,...,0.450053,1.145618e-10,859.536243,860.000707,860.003966,859.995947,283.828118,280.178952,331.886844,332.604837


- 불량률이 가장 높은 배정번호는 128795 가장 낮은 배정번호는 104126

In [59]:
df_104126 = df[df['AN'] == 104126]
df_104126.head()

,AN,DZ1_OP,DZ2_OP,DZ1_TEMP,DZ2_TEMP,CLEAN,HDZ1_OP,HDZ2_OP,HDZ3_OP,HDZ4_OP,HDZ_CP,HDZ_CPM,HDZ1_TEMP,HDZ2_TEMP,HDZ3_TEMP,HDZ4_TEMP,SCZ1_TEMP,SCZ2_TEMP,STZ1_TEMP,STZ2_TEMP
111278,104126,69.4803,26.0150,100.002,99.8174,69.4575,73.2032,46.8250,46.7220,68.8194,0.448316,1.145550e-10,860.158,860.022,860.002,860.000,284.586,280.02,332.017,329.191
111279,104126,71.8280,23.9977,100.685,99.8174,69.4575,73.6174,46.8453,48.2673,70.4043,0.448469,1.147550e-10,860.158,860.022,860.120,859.866,292.796,280.02,332.017,329.191
111280,104126,71.9123,26.5839,100.552,99.8100,69.4464,79.3722,47.1393,48.4703,69.0781,0.448658,1.147390e-10,860.082,860.028,860.110,859.920,292.794,286.58,328.930,329.187
111281,104126,69.8034,27.0136,100.552,99.7485,69.4464,79.3389,47.1366,48.4282,70.6452,0.448882,1.147390e-10,860.082,860.028,860.110,859.858,292.794,286.58,328.930,329.187
111282,104126,69.7929,24.9979,100.614,99.8100,69.4464,79.3082,47.1341,48.3896,69.0614,0.448966,1.147390e-10,860.082,860.028,860.110,859.920,292.794,286.58,328.930,329.187


In [60]:
df_128795 = df[df['AN'] == 128795]
df_128795.head()

,AN,DZ1_OP,DZ2_OP,DZ1_TEMP,DZ2_TEMP,CLEAN,HDZ1_OP,HDZ2_OP,HDZ3_OP,HDZ4_OP,HDZ_CP,HDZ_CPM,HDZ1_TEMP,HDZ2_TEMP,HDZ3_TEMP,HDZ4_TEMP,SCZ1_TEMP,SCZ2_TEMP,STZ1_TEMP,STZ2_TEMP
1440156,128795,63.3972,22.0008,100.002,100.0190,66.0195,89.3817,52.6439,52.0705,67.0273,0.451138,1.145550e-10,860.958,860.022,860.002,860.000,284.586,290.281,332.017,332.423
1440157,128795,63.2470,22.1017,100.783,100.0190,65.9582,82.3504,52.6312,52.0466,67.0272,0.451324,1.145550e-10,861.020,860.022,860.002,860.000,284.586,290.281,332.017,332.423
1440158,128795,65.4958,22.3130,100.844,99.6475,65.9582,84.6438,52.6193,52.0246,67.0820,0.451475,1.145550e-10,860.958,860.022,860.002,860.000,284.586,290.281,332.017,332.597
1440159,128795,65.5671,22.6320,100.774,99.5786,65.9472,82.4049,52.6082,52.0042,68.5948,0.451962,1.145550e-10,860.958,860.022,860.002,860.375,284.586,290.281,331.911,332.657
1440160,128795,63.1781,22.6623,100.835,99.5786,65.8858,84.5844,54.1172,51.9853,68.5920,0.452498,1.145550e-10,860.896,860.286,860.002,860.375,297.396,290.281,331.971,332.597


In [61]:
df_104126.drop('AN', axis=1, inplace=True)
df_128795.drop('AN', axis=1, inplace=True)

In [62]:
# # 표준화
# from sklearn.preprocessing import StandardScaler

# # StandardScaler 객체 생성
# scaler = StandardScaler()

# # 표준화 적용
# standard_104126 = pd.DataFrame(scaler.fit_transform(df_104126), columns=df_104126.columns)
# standard_128795 = pd.DataFrame(scaler.fit_transform(df_128795), columns=df_128795.columns)

### 정규성 검정
- 표본의 수가 30이상이므로 중심극한정리에 의해 정규성을 만족한다고 가정

### 등분산성 검정

In [69]:
for column in df_104126.columns:
    print(f'{column}설비')
    var_test_stat, var_test_p_val = stats.bartlett(df_104126[column], df_128795[column])
    if var_test_p_val > 0.05:
        print(f'두 배정번호 간의 {column}설비는 등분산성을 만족한다.')
        t_statistic, p_value = stats.ttest_ind(df_104126[column], df_128795[column], equal_var=True)
        if p_value < 0.05:
            print(f'두 배정번호 간의 {column}설비데이터는 유의하게 차이가 있다')
        else:
            print(f'두 배정번호 간의 {column}설비데이터는 유의한 차이가 없다')
    else:
        print(f'두 배정번호 간의 {column}설비는 등분산성을 만족하지 않는다.')
        t_statistic, p_value = stats.ttest_ind(df_104126[column], df_128795[column], equal_var=False)
        if p_value < 0.05:
            print(f'두 배정번호 간의 {column}설비데이터는 유의하게 차이가 있다')
        else:
            print(f'두 배정번호 간의 {column}설비데이터는 유의한 차이가 없다')
    print()

DZ1_OP설비
두 배정번호 간의 DZ1_OP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 DZ1_OP설비데이터는 유의하게 차이가 있다

DZ2_OP설비
두 배정번호 간의 DZ2_OP설비는 등분산성을 만족한다.
두 배정번호 간의 DZ2_OP설비데이터는 유의하게 차이가 있다

DZ1_TEMP설비
두 배정번호 간의 DZ1_TEMP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 DZ1_TEMP설비데이터는 유의하게 차이가 있다

DZ2_TEMP설비
두 배정번호 간의 DZ2_TEMP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 DZ2_TEMP설비데이터는 유의하게 차이가 있다

CLEAN설비
두 배정번호 간의 CLEAN설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 CLEAN설비데이터는 유의하게 차이가 있다

HDZ1_OP설비
두 배정번호 간의 HDZ1_OP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 HDZ1_OP설비데이터는 유의하게 차이가 있다

HDZ2_OP설비
두 배정번호 간의 HDZ2_OP설비는 등분산성을 만족한다.
두 배정번호 간의 HDZ2_OP설비데이터는 유의하게 차이가 있다

HDZ3_OP설비
두 배정번호 간의 HDZ3_OP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 HDZ3_OP설비데이터는 유의하게 차이가 있다

HDZ4_OP설비
두 배정번호 간의 HDZ4_OP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 HDZ4_OP설비데이터는 유의하게 차이가 있다

HDZ_CP설비
두 배정번호 간의 HDZ_CP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 HDZ_CP설비데이터는 유의하게 차이가 있다

HDZ_CPM설비
두 배정번호 간의 HDZ_CPM설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 HDZ_CPM설비데이터는 유의하게 차이가 있다

HDZ1_TEMP설비
두 배정번호 간의 HDZ1_TEMP설비는 등분산성을 만족하지 않는다.
두 배정번호 간의 HDZ1_TEMP설비데이터는 유의하게 차이가 있다

HDZ